# LLM as a Judge

In [ ]:
!pip install langchain_openai

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

/Users/hyeonjinho/.pyenv/versions/3.11.6/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


## Single Scoring
- Self-Rewarding Language Models
    - https://arxiv.org/pdf/2401.10020.pdf

In [2]:
# judge_llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0)
judge_llm = ChatOpenAI(model='gpt-4-1106-preview', temperature=0)

In [3]:
class SingleScoringCoT(BaseModel):
    thought: str = Field(description="Step-by-Step Thought Process")
    score: int = Field(description="score, 1~5")

In [4]:
parser = JsonOutputParser(pydantic_object=SingleScoringCoT)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"thought": {"title": "Thought", "description": "Step-by-Step Thought Process", "type": "string"}, "score": {"title": "Score", "description": "score, 1~5", "type": "integer"}}, "required": ["thought", "score"]}\n```'

In [5]:
judge_prompt_template_text = """\
사용자의 질문과 후보 응답에 대한 평가를 다음 기준을 사용하여 5점 척도로 제공해주세요:
1: 응답이 불완전하거나, 모호하거나, 주제에서 벗어났거나, 논란의 여지가 있거나, 사용자가 요청한 것과 정확히 일치하지 않습니다. 예를 들어, 일부 내용이 누락되었거나, 번호 매긴 목록이 처음부터 시작하지 않거나, 서두가 사용자의 질문을 반복합니다. 또는 응답이 다른 사람의 관점에서 그들의 개인 경험(예: 블로그 포스트에서 가져옴)으로 작성되었거나 포럼에서의 답변처럼 보입니다. 또는 홍보 텍스트, 네비게이션 텍스트 또는 기타 관련 없는 정보를 포함합니다.
2: 응답이 사용자의 요청 대부분을 다룹니다. 사용자의 질문에 직접적으로 답하지 않습니다. 예를 들어, 사용자의 질문에 대한 정확한 해결책 대신 고차원적인 방법론만을 제공합니다.
3: 응답이 유용하지만 AI 어시스턴트가 작성한 것이 아닙니다. 사용자의 기본적인 요구 사항을 모두 다룹니다. 완전하고 독립적이지만, 다른 사람의 관점에서 작성된 단점이 있습니다. 예를 들어, 개인 경험이나 의견을 포함하거나, 댓글 섹션 언급이나 소셜 미디어 공유 등이 포함됩니다.
4: AI 어시스턴트의 관점에서 명확하게 사용자의 지시를 다루는 응답입니다. 사용자의 질문이나 지시에 대해 누락되거나 관련 없는 정보 없이 완전하고 명확하며 포괄적인 응답을 제공합니다. 잘 조직되어 있고, 자체 완결적이며, 도움이 되는 어조로 작성되었습니다. 약간의 개선 여지가 있습니다. 예를 들어, 더 간결하고 집중적일 수 있습니다.
5: AI 어시스턴트의 완벽한 답변입니다. 사용자의 질문이나 지시를 해결하기 위해 의도적으로 작성된 것처럼 보이는 명확한 초점을 가지고 있습니다. 고품질의 내용을 제공하며, 해당 분야에서 전문 지식을 보여주며, 매우 잘 작성되었고 논리적이며, 따라하기 쉽고, 매력적이며, 통찰력이 있습니다.

사용자: {question}
<response>{response}</response>
먼저 간략한 이유를 설명한 후 작성하세요. AI 어시스턴트의 스타일로 답변하세요. 최종 점수를 도출하기 위해 기준에 따라 단계별로 생각해보세요.
{format_instructions}
"""

judge_prompt_template = PromptTemplate.from_template(judge_prompt_template_text,
                                                     partial_variables={"format_instructions": format_instructions})
               

In [6]:
judge_chain = judge_prompt_template | judge_llm | parser

In [7]:
question = "123+123"

# response = "글쎄요 잘 모르겠네요"
response = "123+123의 정답은 246입니다."

In [8]:
judge_chain.invoke({"question": question,
                    "response": response})

{'thought': "The response directly answers the user's mathematical query without any unnecessary information or errors. It is concise, accurate, and directly to the point, which aligns with the user's request for a simple arithmetic solution. There is no additional context or explanation, but none is needed for such a straightforward question. The response does not include any personal opinions, external perspectives, or irrelevant content. It is written from the AI assistant's perspective, providing a clear and direct answer to the user's question. Therefore, it meets the criteria for a high score. However, it lacks an element of engagement or additional value that might be provided by explaining the calculation process or offering related educational content, which could elevate the response from being simply correct to being insightful or educational.",
 'score': 5}

## Pairwise Scoring

- https://github.com/lm-sys/FastChat/blob/main/fastchat/llm_judge/data/judge_prompts.jsonl

In [9]:
class PairScoringCoT(BaseModel):
    thought: str = Field(description="Step-by-Step Thought Process")
    assistant_name: str = Field(description="assistant name. 'A' | 'B', if tie, then 'TIE'")

In [10]:
parser = JsonOutputParser(pydantic_object=PairScoringCoT)
format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"thought": {"title": "Thought", "description": "Step-by-Step Thought Process", "type": "string"}, "assistant_name": {"title": "Assistant Name", "description": "assistant name. \'A\' | \'B\', if tie, then \'TIE\'", "type": "string"}}, "required": ["thought", "assistant_name"]}\n```'

In [11]:
judge_prompt_template_text = """\
두 AI 어시스턴트가 제공한 답변의 품질을 공정한 판단으로 평가해주세요.
아래에 표시된 사용자 질문에 대한 답변을 평가할 때, 사용자의 지시를 따르고 사용자의 질문에 더 잘 답한 어시스턴트를 선택해야 합니다.
이 평가는 도움이 되는 정도, 관련성, 정확성, 깊이, 창의성 및 세부 사항의 수준과 같은 요소들을 고려해야 합니다.
두 응답을 비교하며 평가를 시작하고, 간단한 설명을 제공해 주세요. 위치 편향이나 제시된 순서가 결정에 영향을 미치지 않도록 주의하세요.
응답의 길이가 평가에 영향을 미치지 않도록 하세요. 특정 어시스턴트의 이름을 선호하지 않도록 하세요. 가능한 한 객관적이어야 합니다.
설명을 제공한 후, 다음 형식을 엄격하게 따라 최종 판결을 내려주세요
{format_instructions}

[사용자 질문]
{question}

[어시스턴트 A의 답변 시작]
{answer_a}
[어시스턴트 A의 답변 끝]

[어시스턴트 B의 답변 시작]
{answer_b}
[어시스턴트 B의 답변 끝]
"""

judge_prompt_template = PromptTemplate.from_template(judge_prompt_template_text,
                                                     partial_variables={"format_instructions": format_instructions})
               

In [12]:
judge_chain = judge_prompt_template | judge_llm | parser

In [13]:
question = "공룡은 언제까지 살았어?"
answer_a = "공룡은 약 2억 3000만 년 전, 쥐라기와 백악기를 포함한 중생대 시대에 번성했으며, 대략 6600만 년 전에 대멸종 사건으로 인해 대부분이 멸종했습니다."
answer_b = "몰라요"

In [14]:
judge_chain.invoke({"question": question,
                    "answer_a": answer_a,
                    "answer_b": answer_b})

{'thought': "Assistant A provided a detailed and accurate response, mentioning the time period when dinosaurs thrived and the mass extinction event that led to their demise. Assistant B, on the other hand, simply responded with '몰라요' (I don't know), which is not helpful and does not answer the user's question. Therefore, Assistant A gave a better answer.",
 'assistant_name': 'A'}